In [8]:
from pprint import pprint

In [102]:
import itertools
from collections import namedtuple, defaultdict
from dataclasses import dataclass

In [222]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
import vg

In [ ]:


In [21]: @dataclass
    ...: class InventoryItem:
    ...:     '''Class for keeping track of an item in inventory.'''
    ...:     name: str
    ...:     unit_price: float
    ...:     quantity_on_hand: int = 0
    ...: 
    ...:     def total_cost(self) -> float:
    ...:         return self.unit_price * self.quantity_on_hand
    ...:    


In [113]:
@dataclass
class Scanner:
    _id: int
    pings: list = None
    pairs: list = None

In [234]:
Ping = namedtuple('Ping', ['x', 'y', 'z'])
Pair = namedtuple('Pair', ['a', 'b', 'f'])
Fingerprint = namedtuple('Fingerprint', ['pair', 'scanner'])

In [6]:
EXAMPLE1 = """--- scanner 0 ---
0,2
4,1
3,3

--- scanner 1 ---
-1,-1
-5,0
-2,1"""

In [175]:
EXAMPLE2 = """--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7

--- scanner 0 ---
1,-1,1
2,-2,2
3,-3,3
2,-1,3
-5,4,-6
-8,-7,0

--- scanner 0 ---
-1,-1,-1
-2,-2,-2
-3,-3,-3
-1,-3,-2
4,6,5
-7,0,8

--- scanner 0 ---
1,1,-1
2,2,-2
3,3,-3
1,3,-2
-4,-6,5
7,0,8

--- scanner 0 ---
1,1,1
2,2,2
3,3,3
3,1,2
-6,-4,-5
0,7,-8"""



In [236]:
def parse_pings(txt):
    scanners = []
    for line in txt.splitlines():
        if line.startswith('--- scanner '):
            pings = []
            scanners.append(Scanner(_id=len(scanners), pings=pings, pairs=None))
            continue

        if not line.rstrip():
            continue

        vals = [int(v) for v in line.rstrip().split(',')]
        if len(vals) == 2:
            vals += [0]
        ping = Ping(*vals)
        pings.append(ping)
    return scanners

In [238]:
def calc_pair_fingerprints(pings):
    pairs = []
    for p1, p2 in itertools.combinations(pings, 2):
        dx, dy, dz = p1.x - p2.x, p1.y - p2.y, p1.z - p2.z
        f = (dx**2 + dy**2 + dz**2,
             abs(dx) + abs(dy) + abs(dz),
             min(abs(dx), abs(dy), abs(dz)),
             max(abs(dx), abs(dy), abs(dz)))
        pair = Pair(p1, p2, f)
        pairs.append(pair)
    return pairs

In [240]:
# txt = EXAMPLE1
txt = EXAMPLE2

scanners = parse_pings(txt)

fingerprints = defaultdict(list)
for scanner in scanners:
    pairs = calc_pair_fingerprints(scanner.pings)
    for pair in pairs:
        fingerprints[pair.f].append(Fingerprint(pair, scanner._id))

pprint(scanners)
print('')
pprint(fingerprints)

[Scanner(_id=0, pings=[Ping(x=-1, y=-1, z=1), Ping(x=-2, y=-2, z=2), Ping(x=-3, y=-3, z=3), Ping(x=-2, y=-3, z=1), Ping(x=5, y=6, z=-4), Ping(x=8, y=0, z=7)], pairs=None),
 Scanner(_id=1, pings=[Ping(x=1, y=-1, z=1), Ping(x=2, y=-2, z=2), Ping(x=3, y=-3, z=3), Ping(x=2, y=-1, z=3), Ping(x=-5, y=4, z=-6), Ping(x=-8, y=-7, z=0)], pairs=None),
 Scanner(_id=2, pings=[Ping(x=-1, y=-1, z=-1), Ping(x=-2, y=-2, z=-2), Ping(x=-3, y=-3, z=-3), Ping(x=-1, y=-3, z=-2), Ping(x=4, y=6, z=5), Ping(x=-7, y=0, z=8)], pairs=None),
 Scanner(_id=3, pings=[Ping(x=1, y=1, z=-1), Ping(x=2, y=2, z=-2), Ping(x=3, y=3, z=-3), Ping(x=1, y=3, z=-2), Ping(x=-4, y=-6, z=5), Ping(x=7, y=0, z=8)], pairs=None),
 Scanner(_id=4, pings=[Ping(x=1, y=1, z=1), Ping(x=2, y=2, z=2), Ping(x=3, y=3, z=3), Ping(x=3, y=1, z=2), Ping(x=-6, y=-4, z=-5), Ping(x=0, y=7, z=-8)], pairs=None)]

defaultdict(<class 'list'>,
            {(2, 2, 0, 1): [Fingerprint(pair=Pair(a=Ping(x=-2, y=-2, z=2), b=Ping(x=-2, y=-3, z=1), f=(2, 2, 0, 1)),

In [242]:
list(fingerprints.values())

[[Fingerprint(pair=Pair(a=Ping(x=-1, y=-1, z=1), b=Ping(x=-2, y=-2, z=2), f=(3, 3, 1, 1)), scanner=0),
  Fingerprint(pair=Pair(a=Ping(x=-2, y=-2, z=2), b=Ping(x=-3, y=-3, z=3), f=(3, 3, 1, 1)), scanner=0),
  Fingerprint(pair=Pair(a=Ping(x=1, y=-1, z=1), b=Ping(x=2, y=-2, z=2), f=(3, 3, 1, 1)), scanner=1),
  Fingerprint(pair=Pair(a=Ping(x=2, y=-2, z=2), b=Ping(x=3, y=-3, z=3), f=(3, 3, 1, 1)), scanner=1),
  Fingerprint(pair=Pair(a=Ping(x=-1, y=-1, z=-1), b=Ping(x=-2, y=-2, z=-2), f=(3, 3, 1, 1)), scanner=2),
  Fingerprint(pair=Pair(a=Ping(x=-2, y=-2, z=-2), b=Ping(x=-3, y=-3, z=-3), f=(3, 3, 1, 1)), scanner=2),
  Fingerprint(pair=Pair(a=Ping(x=1, y=1, z=-1), b=Ping(x=2, y=2, z=-2), f=(3, 3, 1, 1)), scanner=3),
  Fingerprint(pair=Pair(a=Ping(x=2, y=2, z=-2), b=Ping(x=3, y=3, z=-3), f=(3, 3, 1, 1)), scanner=3),
  Fingerprint(pair=Pair(a=Ping(x=1, y=1, z=1), b=Ping(x=2, y=2, z=2), f=(3, 3, 1, 1)), scanner=4),
  Fingerprint(pair=Pair(a=Ping(x=2, y=2, z=2), b=Ping(x=3, y=3, z=3), f=(3, 3, 1,

In [185]:
candidates = fingerprints[12]

In [186]:
candidates

[Fingerprint(pair=Pair(a=Ping(x=-1, y=-1, z=1), b=Ping(x=-3, y=-3, z=3), dm=12), scanner=0),
 Fingerprint(pair=Pair(a=Ping(x=1, y=-1, z=1), b=Ping(x=3, y=-3, z=3), dm=12), scanner=1),
 Fingerprint(pair=Pair(a=Ping(x=-1, y=-1, z=-1), b=Ping(x=-3, y=-3, z=-3), dm=12), scanner=2),
 Fingerprint(pair=Pair(a=Ping(x=1, y=1, z=-1), b=Ping(x=3, y=3, z=-3), dm=12), scanner=3),
 Fingerprint(pair=Pair(a=Ping(x=1, y=1, z=1), b=Ping(x=3, y=3, z=3), dm=12), scanner=4)]

In [188]:
f1, f2 = candidates[:2]

In [189]:
f = f1
a,b = f.pair.a, f.pair.b
d1 = a.x - b.x, a.y - b.y, a.z - b.z

In [190]:
f = f2
a,b = f.pair.a, f.pair.b
d2 = a.x - b.x, a.y - b.y, a.z - b.z

In [191]:
d1 = np.array(d1)

In [192]:
d2 = np.array(d2)

In [193]:
d1

array([ 2,  2, -2])

In [194]:
d2

array([-2,  2, -2])

In [252]:
cosine_similarity([d1, d2])

array([[1.        , 0.33333333],
       [0.33333333, 1.        ]])

In [253]:
np.linalg.det(cosine_similarity([d1, d2]))

0.8888888888888893

In [254]:
np.cross(d1, d2)

array([0, 8, 8])

In [255]:
d1

array([ 2,  2, -2])

In [256]:
d2

array([-2,  2, -2])

In [257]:
len(rot)

24

In [259]:
match(tuple(d1), tuple(d2))

TypeError: 'numpy.int64' object is not iterable

In [249]:
def shift(y, z):
    sy = {tuple(x) for x in y}
    for a in y:
        for b in z:
            s = a-b
            ov = len(sy.intersection(tuple(x+s) for x in z))
            if ov >= 12: return s
    return None

def match(y, z):
    for r in rot:
        rz = [r.dot(x) for x in z]
        s = shift(y, rz)
        if s is None: continue
        else: return s, r
    return None

In [245]:
rot = []
for x in [-1, 1]:
    for y in [-1, 1]:
        for z in [-1, 1]:
            for q in itertools.permutations([[x,0,0], [0,y,0], [0,0,z]]):
                m = np.array(q)
                if np.linalg.det(m) == 1:
                    rot.append(m)
rot

[array([[-1,  0,  0],
        [ 0,  0, -1],
        [ 0, -1,  0]]),
 array([[ 0, -1,  0],
        [-1,  0,  0],
        [ 0,  0, -1]]),
 array([[ 0,  0, -1],
        [ 0, -1,  0],
        [-1,  0,  0]]),
 array([[-1,  0,  0],
        [ 0, -1,  0],
        [ 0,  0,  1]]),
 array([[ 0, -1,  0],
        [ 0,  0,  1],
        [-1,  0,  0]]),
 array([[ 0,  0,  1],
        [-1,  0,  0],
        [ 0, -1,  0]]),
 array([[-1,  0,  0],
        [ 0,  1,  0],
        [ 0,  0, -1]]),
 array([[ 0,  1,  0],
        [ 0,  0, -1],
        [-1,  0,  0]]),
 array([[ 0,  0, -1],
        [-1,  0,  0],
        [ 0,  1,  0]]),
 array([[-1,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  0]]),
 array([[ 0,  1,  0],
        [-1,  0,  0],
        [ 0,  0,  1]]),
 array([[ 0,  0,  1],
        [ 0,  1,  0],
        [-1,  0,  0]]),
 array([[ 1,  0,  0],
        [ 0, -1,  0],
        [ 0,  0, -1]]),
 array([[ 0, -1,  0],
        [ 0,  0, -1],
        [ 1,  0,  0]]),
 array([[ 0,  0, -1],
        [ 1,  0,  0],
    

In [261]:
def determinant(m):
  det = 0
  for i in range(3):
    det += (m[0][i]*(m[1][(i+1)%3]*m[2][(i+2)%3] - m[1][(i+2)%3]*m[2][(i+1)%3]))
  return det

In [262]:
triples=((1,0,0),(0,1,0),(0,0,1),(-1,0,0),(0,-1,0),(0,0,-1))
orientations = [(x,y,z) for x in triples for y in triples for z in triples if determinant((x,y,z)) == 1]

In [263]:
triples

((1, 0, 0), (0, 1, 0), (0, 0, 1), (-1, 0, 0), (0, -1, 0), (0, 0, -1))

In [267]:
orientations

[((1, 0, 0), (0, 1, 0), (0, 0, 1)),
 ((1, 0, 0), (0, 0, 1), (0, -1, 0)),
 ((1, 0, 0), (0, -1, 0), (0, 0, -1)),
 ((1, 0, 0), (0, 0, -1), (0, 1, 0)),
 ((0, 1, 0), (1, 0, 0), (0, 0, -1)),
 ((0, 1, 0), (0, 0, 1), (1, 0, 0)),
 ((0, 1, 0), (-1, 0, 0), (0, 0, 1)),
 ((0, 1, 0), (0, 0, -1), (-1, 0, 0)),
 ((0, 0, 1), (1, 0, 0), (0, 1, 0)),
 ((0, 0, 1), (0, 1, 0), (-1, 0, 0)),
 ((0, 0, 1), (-1, 0, 0), (0, -1, 0)),
 ((0, 0, 1), (0, -1, 0), (1, 0, 0)),
 ((-1, 0, 0), (0, 1, 0), (0, 0, -1)),
 ((-1, 0, 0), (0, 0, 1), (0, 1, 0)),
 ((-1, 0, 0), (0, -1, 0), (0, 0, 1)),
 ((-1, 0, 0), (0, 0, -1), (0, -1, 0)),
 ((0, -1, 0), (1, 0, 0), (0, 0, 1)),
 ((0, -1, 0), (0, 0, 1), (-1, 0, 0)),
 ((0, -1, 0), (-1, 0, 0), (0, 0, -1)),
 ((0, -1, 0), (0, 0, -1), (1, 0, 0)),
 ((0, 0, -1), (1, 0, 0), (0, -1, 0)),
 ((0, 0, -1), (0, 1, 0), (1, 0, 0)),
 ((0, 0, -1), (-1, 0, 0), (0, 1, 0)),
 ((0, 0, -1), (0, -1, 0), (-1, 0, 0))]